In [1]:
import torch


In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
y = x + 2
print(y)


tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
print(y.grad_fn)

In [6]:
z = y*y*3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [7]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [8]:
out.backward()

In [9]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [10]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([   -5.2940,  -538.7870, -1071.1195], grad_fn=<MulBackward0>)


In [11]:
x = torch.randn(3, requires_grad=True)
y = x * 2
print(y)
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 0.3300, -1.5903,  3.3435], grad_fn=<MulBackward0>)
tensor([ 168.9812, -814.2557, 1711.8734], grad_fn=<MulBackward0>)


In [12]:
x = torch.randn(3, requires_grad=True)
y = x * 2
print(y)
y.data.norm()
print(y)


tensor([ 1.0336, -1.5695, -0.6768], grad_fn=<MulBackward0>)
tensor([ 1.0336, -1.5695, -0.6768], grad_fn=<MulBackward0>)


In [14]:
x = torch.randn(3, requires_grad=True)

y = x * 2
print(y)
y.data.norm()
print(y)
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([2.3611, 2.2934, 0.9235], grad_fn=<MulBackward0>)
tensor([2.3611, 2.2934, 0.9235], grad_fn=<MulBackward0>)
tensor([1208.8994, 1174.2330,  472.8571], grad_fn=<MulBackward0>)


In [15]:
x = torch.randn(3, requires_grad=True)

y = x * 2
print(y)
y.data.norm() < 1000
print(y)
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-1.0654,  2.6875, -1.7169], grad_fn=<MulBackward0>)
tensor([-1.0654,  2.6875, -1.7169], grad_fn=<MulBackward0>)
tensor([-545.5050, 1376.0189, -879.0717], grad_fn=<MulBackward0>)


In [16]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)

In [17]:
print(v)

tensor([1.0000e-01, 1.0000e+00, 1.0000e-04])


In [19]:
y.backward(v)

In [20]:
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [21]:
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
False


In [22]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [24]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [25]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1014,  0.0064,  0.0236,  0.0409,  0.0567,  0.0141, -0.0658,  0.0256,
         -0.0724, -0.0314]], grad_fn=<AddmmBackward>)


In [26]:
print(params)


[Parameter containing:
tensor([[[[ 0.0632,  0.0453, -0.0489],
          [-0.2559,  0.0908,  0.3287],
          [ 0.2660, -0.2663, -0.1560]]],


        [[[-0.1643,  0.0426, -0.0159],
          [-0.3157, -0.2607,  0.1475],
          [ 0.2444,  0.2989,  0.2903]]],


        [[[ 0.2286, -0.0669,  0.2363],
          [-0.3136, -0.1829,  0.0984],
          [-0.0921,  0.0623,  0.1353]]],


        [[[ 0.2301, -0.3017,  0.0786],
          [-0.0649,  0.2608,  0.2206],
          [ 0.0581, -0.0880,  0.1821]]],


        [[[-0.1685, -0.2751, -0.0428],
          [-0.2378, -0.0657, -0.0214],
          [ 0.2211, -0.0367, -0.0107]]],


        [[[ 0.3211,  0.0787, -0.2549],
          [ 0.2323,  0.2537,  0.0494],
          [ 0.0216,  0.2494,  0.2275]]]], requires_grad=True), Parameter containing:
tensor([-0.1725,  0.0083, -0.1488,  0.2210,  0.1619, -0.1759],
       requires_grad=True), Parameter containing:
tensor([[[[-5.4536e-03, -7.7456e-02,  2.8291e-03],
          [ 1.1570e-01,  2.4970e-02, -1.0426e

        -0.0633,  0.0245], requires_grad=True)]


In [27]:
net.zero_grad()

In [28]:
out.backward(torch.randn(1, 10))

In [29]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1321, grad_fn=<MseLossBackward>)


In [30]:
print(loss.grad_fun)


AttributeError: 'Tensor' object has no attribute 'grad_fun'

In [32]:
input

tensor([[[[-0.1156,  0.0084,  0.5280,  ..., -0.4433,  1.3025, -1.4785],
          [ 0.7296, -0.3683,  0.3157,  ..., -1.1031,  1.4163, -0.1587],
          [-1.3868,  1.3393,  0.9328,  ...,  1.4711,  0.2601, -0.2581],
          ...,
          [-0.6008, -1.8554, -0.3654,  ...,  1.2106, -2.0185, -1.6612],
          [-0.0393, -0.9876, -1.8161,  ...,  0.1347,  1.2538, -0.0979],
          [ 1.1202,  1.4701,  1.0926,  ..., -1.1245,  0.3872, -0.2298]]]])

In [33]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [34]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0045, -0.0130, -0.0134, -0.0136,  0.0177,  0.0071])


In [35]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [36]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update